In [92]:
import os 
import os.path as osp
import numpy as np 
import cv2
import open3d as o3d
import json
import pickle

#the other imports from the local stuff
import sys

ws_dir = '/local/home/ekoller/BT'
print(ws_dir)
sys.path.append(ws_dir)
from utils import scan3r,visualisation

/local/home/ekoller/BT


In [93]:
#reading in the necessary data
data_dir ='/local/home/ekoller/R3Scan'
scenes_dir = '/local/home/ekoller/R3Scan/scenes'
scan_id= "38770c95-86d7-27b8-8717-3485b411ddc7" #is reference scan  since it is a reference scan everything shouls be correctly hit
frame_number = "000015"
img_width = 960
img_height = 540

curr_scan_id = "10b1795b-3938-2467-88fe-b10cad8913f8" #is ref 1d2f8510-d757-207c-8c48-3684433860e1
new_scan_id =  "1d2f8510-d757-207c-8c48-3684433860e1" #is rescan 9c27de56-6184-2cda-8196-591957b6387d

#the original meshes are given in the file  'labels.instances.annotated.v2.ply


This code segment does the following: given a mesh of the scene, create the camera rays and intersect them with the mesh- For every camera pose do the following: the camera pose is given in the original frame of the scan: if reference scan in reference coordinates, if rescan in rescan coordinates. To do the computations the designdecision is the following:  use the aligned version of the rescan so that the mesh is in the reference coordinate system. in this coordinate system you can then compare the two things. Compare the object of the intersection of the current mesh with the one of the new mesh


Logic for the try out data: in the gt_tmp we have for every scene a folder with the frame of the ground truth projection
in the projection file we have the following way of storing things projection/curr_scene_id/(color/global_id/obj_id)/projection_pose_in_new_scene_id_000033

In [153]:
#the function which actually does the intersection part
def segmentResult(scene, intrinsics, extrinsics, width, height,
                      mesh_triangles, num_triangles, colors, obj_ids, global_ids):
        
        #create the rays to shoot in the scene
        rays = o3d.t.geometry.RaycastingScene.create_rays_pinhole(
            intrinsic_matrix = intrinsics.astype(np.float64),
            extrinsic_matrix = extrinsics.astype(np.float64),
            width_px = width, height_px = height
        )
        
       
        #find the intersections of the rays
        ans = scene.cast_rays(rays)
        hit_triangles_ids = ans['primitive_ids'].numpy()
        hit_triangles_ids_valid_masks = (hit_triangles_ids<num_triangles)
        hit_triangles_ids_valid = hit_triangles_ids[hit_triangles_ids_valid_masks]
        hit_triangles_valid = mesh_triangles[hit_triangles_ids_valid]
        hit_points_ids_valid = hit_triangles_valid[:,0]
        
        #create resulting maps initialized to 0
        color_map = np.zeros((height,width,3), dtype=np.uint8)
        obj_id_map = np.zeros((height,width), dtype=np.uint8)
        global_id_map = np.zeros((height,width), dtype=np.uint8)

        #go through the array and add the corresponding values
        color_map[hit_triangles_ids_valid_masks] = colors[hit_points_ids_valid]
        obj_id_map[hit_triangles_ids_valid_masks] = obj_ids[hit_points_ids_valid]
        global_id_map[hit_triangles_ids_valid_masks] = global_ids[hit_points_ids_valid]
        
        return color_map, obj_id_map, global_id_map

In [165]:
#given a scan id and a corresponding frame number in the same coordinate system sanity check if the same as groundtruth
def project_mesh_gt_generation(data_dir,scenes_dir, scan_id, frame_number):
        # get related files
        mesh_file = osp.join(scenes_dir, scan_id, "labels.instances.align.annotated.v2.ply") #exists for bot reference and rescan
        

        # get img info and camera intrinsics 
        camera_info = scan3r.load_intrinsics(scenes_dir, scan_id)
        intrinsics = camera_info['intrinsic_mat']
        img_width = int(camera_info['width'])
        img_height = int(camera_info['height'])
        
        # load labels
        plydata_npy = np.load(osp.join(scenes_dir, scan_id, "data.npy"), allow_pickle=True)
        
        #print the names of the datafields
        # if plydata_npy.dtype.names:
        #         for name in plydata_npy.dtype.names:
        #              print(f"Field: {name}")
        #              print(plydata_npy[name])
        # else:
        #     print(plydata_npy)


        #get the corresponding ids
        obj_labels = plydata_npy['objectId']
        global_labels = plydata_npy['globalId']
    
      
    
        # load mesh and scene
        mesh = o3d.io.read_triangle_mesh(mesh_file)
        mesh_triangles = np.asarray(mesh.triangles)
        #also load the colour
        colors = np.asarray(mesh.vertex_colors)*255.0
        colors = colors.round()
        num_triangles = mesh_triangles.shape[0]
        scene = o3d.t.geometry.RaycastingScene()
        scene.add_triangles(o3d.t.geometry.TriangleMesh.from_legacy(mesh))
        
        # get frame_indexes
        frame_idxs = frame_number
        poses = scan3r.load_pose(scenes_dir, scan_id, frame_number)
        
        # project 3D model using rays
        obj_id_imgs = {}
        global_id_imgs = {}
        color_imgs = {}
        for idx in range(0, len(poses), 1):
            #frame_idx = frame_idxs[idx] #adjusted for single use in this jupiter notebook
            frame_idx = frame_idxs
            #img_pose = poses[idx] #adjusted for single use in this jupiter notebook
            img_pose = poses # camera to rescan 0
        
            extrinsic = np.linalg.inv(img_pose) #rescan to camera  1
         
            
            if scan3r.is_rescan(data_dir, scan_id):
                 #need to adjust the extrinsic matrix to the reference coordinate system
                #get the path to the matricies of each scan_id for transformation of rescan to reference
                path = osp.join(data_dir,"files", "3RScan.json")
                ref2_rescan_all_id = scan3r.read_transform_mat(path)
                ref2rescan = ref2_rescan_all_id[scan_id] #2
                rescan2ref = np.linalg.inv(ref2rescan) #3
                
 
                #multiply such that we get transformation of camera to reference scan
                cam2ref =     extrinsic * rescan2ref.transpose()
            
                #extrinsic = np.linalg.inv(cam2ref)
                #convert
                extrinsic = np.asarray(cam2ref)
               

              
            
            #get the mappings for the image
            color_map, obj_id_map, global_id_map = segmentResult(
                scene, intrinsics, extrinsic, img_width, img_height, 
                mesh_triangles, num_triangles, colors, obj_labels, global_labels
            )
            obj_id_imgs[frame_idx] = obj_id_map
            global_id_imgs[frame_idx] = global_id_map
            color_imgs[frame_idx] = color_map
            
        #create directories (here tmp to try out & save 
        save_scan_color_dir = osp.join(data_dir, "gt_tmp", scan_id, "color")
        save_scan_obj_dir = osp.join(data_dir, "gt_tmp", scan_id, "obj_id")
        save_scan_global_dir = osp.join(data_dir, "gt_tmp", scan_id, "global_id")

        #make sure the directories exist
        for dir_path in [save_scan_color_dir, save_scan_obj_dir, save_scan_global_dir]:
            try:
                os.makedirs(dir_path, exist_ok=True)
            except Exception as e:
                print(f"Failed to create directory {dir_path}: {e}")

        
        for frame_idx in obj_id_imgs:
            obj_id_img = obj_id_imgs[frame_idx]
            global_id_img = global_id_imgs[frame_idx]
            color_img = color_imgs[frame_idx]
            
            
            img_name = "gt_frame-"+str(frame_idx)+".jpg"
            obj_id_img_file = osp.join(save_scan_obj_dir, img_name)
            global_img_file = osp.join(save_scan_global_dir, img_name)
            color_img_file = osp.join(save_scan_color_dir, img_name)
            success_obj = cv2.imwrite(obj_id_img_file, obj_id_img)
            success_global = cv2.imwrite(global_img_file, global_id_img)
            success_color = cv2.imwrite(color_img_file, color_img)

            #also vusually double check if the images are correct
            # cv2.imshow("object_id", obj_id_img)
            # cv2.imshow("global_id", global_id_img)
            # cv2.imshow("color", color_img)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()





In [155]:
#try out the ground truth generation
project_mesh_gt_generation(data_dir,scenes_dir, scan_id, frame_number)


In [215]:
#given a current scene and a new scene: the reference scene is in the reference coordinate system, in this we will project the current camera frame & perform
#raytracing based on the new camera pose onto the current mesh
#frame number is for the new_scan_id
#returns numpy arrays of object_id, color, global_id
def project_mesh_diff(data_dir, scenes_dir,curr_scan_id, new_scan_id, frame_number):

    """
    Read/prepare the data of the current mesh (mesh, colours, objid, globalid)
    """
    #get the current coordinate system the reference coordinate system
    mesh_file = osp.join(scenes_dir, curr_scan_id, "labels.instances.align.annotated.v2.ply") #exists for bot reference and rescan
        


    # load labels based on current mesh "status quo"
    plydata_npy = np.load(osp.join(scenes_dir, curr_scan_id, "data.npy"), allow_pickle=True)
      
    obj_labels = plydata_npy['objectId']
    global_labels = plydata_npy['globalId']
    

    # load curr mesh and scene
    mesh = o3d.io.read_triangle_mesh(mesh_file)
    mesh_triangles = np.asarray(mesh.triangles)
    #also load the colour
    colors = np.asarray(mesh.vertex_colors)*255.0
    colors = colors.round()
    num_triangles = mesh_triangles.shape[0]
    scene = o3d.t.geometry.RaycastingScene()
    scene.add_triangles(o3d.t.geometry.TriangleMesh.from_legacy(mesh))


    """
    Read/prepare the data of the new mesh (pose, intrinsics)
    """
    # get img info and camera intrinsics of the new img
    camera_info = scan3r.load_intrinsics(scenes_dir, new_scan_id)
    intrinsics = camera_info['intrinsic_mat']
    img_width = int(camera_info['width'])
    img_height = int(camera_info['height'])

    # get frame_indexes and pose of the new
    frame_idxs = frame_number
    poses = scan3r.load_pose(scenes_dir, new_scan_id, frame_number)


    """
    Do the intersection
    """

    # project 3D model using rays
    obj_id_imgs = {}
    global_id_imgs = {}
    color_imgs = {}
    for idx in range(0, len(poses), 1):
        #frame_idx = frame_idxs[idx] #adjusted for single use in this jupiter notebook
        frame_idx = frame_idxs
        #img_pose = poses[idx] #adjusted for single use in this jupiter notebook
        img_pose = poses
        extrinsic = np.linalg.inv(img_pose)

        if scan3r.is_rescan(data_dir, new_scan_id):
            #need to adjust the extrinsic matrix to the reference coordinate system
            #get the path to the matricies of each scan_id for transformation of rescan to reference
            path = osp.join(data_dir,"files", "3RScan.json")
            ref2_rescan_all_id = scan3r.read_transform_mat(path)
            ref2rescan = ref2_rescan_all_id[new_scan_id] #2
            rescan2ref = np.linalg.inv(ref2rescan) #3
            

            #multiply such that we get transformation of camera to reference scan
            cam2ref =     extrinsic * rescan2ref.transpose()
    
            #convert
            extrinsic = np.asarray(cam2ref)




        #get the mappings for the image
        color_map, obj_id_map, global_id_map = segmentResult(
            scene, intrinsics, extrinsic, img_width, img_height, 
            mesh_triangles, num_triangles, colors, obj_labels, global_labels
        )
        obj_id_imgs[frame_idx] = obj_id_map
        global_id_imgs[frame_idx] = global_id_map
        color_imgs[frame_idx] = color_map

   
        
    #create directories (here tmp to try out & save 
    save_scan_color_dir = osp.join(data_dir, "proj", curr_scan_id, "color")
    save_scan_obj_dir = osp.join(data_dir, "proj", curr_scan_id, "obj_id")
    save_scan_global_dir = osp.join(data_dir, "proj", curr_scan_id, "global_id")

    #make sure the directories exist
    for dir_path in [save_scan_color_dir, save_scan_obj_dir, save_scan_global_dir]:
        try:
            os.makedirs(dir_path, exist_ok=True)
            
        except Exception as e:
            print(f"Failed to create directory {dir_path}: {e}")

    
    for frame_idx in obj_id_imgs:
        obj_id_img = obj_id_imgs[frame_idx]
        global_id_img = global_id_imgs[frame_idx]
        color_img = color_imgs[frame_idx]

        
        img_name = "projection_pose_in_"+ str(new_scan_id)+"_"+str(frame_idx)+".jpg"
        obj_id_img_file = osp.join(save_scan_obj_dir, img_name)
        global_img_file = osp.join(save_scan_global_dir, img_name)
        color_img_file = osp.join(save_scan_color_dir, img_name)
        success_obj = cv2.imwrite(obj_id_img_file, obj_id_img)
        success_global = cv2.imwrite(global_img_file, global_id_img)
        success_color = cv2.imwrite(color_img_file, color_img)

        #also vusually double check if the images are correct
        # cv2.imshow("object_id", obj_id_img)
        # cv2.imshow("global_id", global_id_img)
        # cv2.imshow("color", color_img)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()   

    
    

In [216]:
project_mesh_diff(data_dir, scenes_dir,curr_scan_id, new_scan_id, frame_number)

In [167]:
#this code returns the colour segmentation of the gt projection on pixelwise level
def load_gt_color_annos(data_dir, scan_id, skip=None):
    anno_imgs = {}
    frame_idxs = scan3r.load_frame_idxs(osp.join(data_dir, "scenes"), scan_id, skip)
    anno_folder = osp.join(data_dir, "files", 'gt_projection/color', scan_id)
    for frame_idx in frame_idxs:
        anno_color_file = osp.join(anno_folder, "frame-{}.jpg".format(frame_idx))
        anno_color = cv2.imread(anno_color_file, cv2.IMREAD_UNCHANGED)
        anno_imgs[frame_idx] = anno_color
    return anno_imgs

In [168]:
#this code returns the object_id segmentation of the gt projection on pixelwise level
def load_gt_obj_id_annos(data_dir, scan_id, skip=None):
    anno_imgs = {}
    frame_idxs = scan3r.load_frame_idxs(osp.join(data_dir, "scenes"), scan_id, skip)
    anno_folder = osp.join(data_dir, "files", 'gt_projection/obj_id', scan_id)
    for frame_idx in frame_idxs:
        anno_obj_file = osp.join(anno_folder, "frame-{}.jpg".format(frame_idx))
        anno_obj = cv2.imread(anno_obj_file, cv2.IMREAD_UNCHANGED)
        anno_imgs[frame_idx] = anno_obj
    return anno_imgs

In [169]:
#this code returns the colour segmentation of the gt projection on pixelwise level
def load_gt_gloabl_id_annos(data_dir, scan_id, skip=None):
    anno_imgs = {}
    frame_idxs = scan3r.load_frame_idxs(osp.join(data_dir, "scenes"), scan_id, skip)
    anno_folder = osp.join(data_dir, "files", 'gt_projection/global_id', scan_id)
    for frame_idx in frame_idxs:
        anno_global_file = osp.join(anno_folder, "frame-{}.jpg".format(frame_idx))
        anno_global = cv2.imread(anno_global_file, cv2.IMREAD_UNCHANGED)
        anno_imgs[frame_idx] = anno_global
    return anno_imgs

In [222]:
#this function gives the transformation on a pixelwise level between the current mesh and the new mesh
#this method goes over the projection and the semantically segmented image provided as input from (new_scan_id)
#it stores the mask following the logic described above
def get_transformation_mask(data_dir, scenes_dir,curr_scan_id, new_scan_id, frame_number):

    #the ground truth of the new scan id must exist already
    #project_mesh_gt_generation(data_dir,scenes_dir, new_scan_id, frame_number)
    


    #get the information of the rgb image of new_scan_id based on the ground truth (this code is for after the ground truth has beeen generated)
    gt_colors = load_gt_color_annos(data_dir,new_scan_id)
    gt_obj_ids = load_gt_obj_id_annos(data_dir, new_scan_id)
    gt_global_ids = load_gt_gloabl_id_annos(data_dir,new_scan_id)

    #get the infor for the specific frame number
    gt_color = gt_colors[frame_number]
    gt_obj_id = gt_obj_ids[frame_number]
    gt_global_id= gt_global_ids[frame_number]

    # scan_color_file = osp.join(data_dir, "gt_tmp", new_scan_id, "color","gt_frame-{}.jpg".format(frame_number))
    # scan_obj_file = osp.join(data_dir, "gt_tmp", new_scan_id, "obj_id","gt_frame-{}.jpg".format(frame_number))
    # scan_global_file = osp.join(data_dir, "gt_tmp", new_scan_id, "global_id","gt_frame-{}.jpg".format(frame_number))

    # gt_color = cv2.imread(scan_color_file, cv2.IMREAD_UNCHANGED)
    # gt_obj_id = cv2.imread(scan_obj_file, cv2.IMREAD_UNCHANGED)
    # gt_global_id= cv2.imread(scan_global_file, cv2.IMREAD_UNCHANGED)


    """
    until here the dimensions and entries are as excpected, however not something is wrong with the way the matrix is saved and filled
    """
    #get the rayintersection of the mesh based on the new computations
    #compute the intersections and maps
    project_mesh_diff(data_dir, scenes_dir,curr_scan_id, new_scan_id, frame_number)
    color_path = osp.join(data_dir,"proj",curr_scan_id, "color","projection_pose_in_{}_{}.jpg".format(new_scan_id,frame_number))
    obj_path = osp.join(data_dir,"proj",curr_scan_id, "obj_id","projection_pose_in_{}_{}.jpg".format(new_scan_id,frame_number))
    gloabl_path = osp.join(data_dir,"proj",curr_scan_id, "global_id","projection_pose_in_{}_{}.jpg".format(new_scan_id,frame_number))

    #access the files
    pj_color = cv2.imread(color_path, cv2.IMREAD_UNCHANGED)
    pj_obj_id = cv2.imread(obj_path, cv2.IMREAD_UNCHANGED)
    pj_global_id = cv2.imread(gloabl_path, cv2.IMREAD_UNCHANGED)

   

    img_height, img_width, channels = pj_color.shape
    #iterate through the whole image 
    curr_res_color = np.full_like(gt_color, 0)
    curr_res_obj_id = np.full((img_height,img_width,1), 0)
    curr_res_global_id = np.full((img_height,img_width,1), 0)
   

    new_res_color = np.full_like(gt_color, 0)
    new_res_obj_id = np.full((img_height,img_width,1), 0)
    new_res_global_id = np.full((img_height,img_width,1), 0)

    #make the mask: originally black: wherever there is a difference take the value of the new_scan_id gt

    

   
    for i in range(img_height):
        for j in range(img_width):
            if (pj_obj_id[i][j] != gt_obj_id[i][j]):
                #the new values in the modified area
                new_res_color[i][j] = gt_color[i][j]
                new_res_obj_id[i][j] = gt_obj_id[i][j]
                new_res_global_id[i][j] = gt_global_id[i][j]

                #the values which are impacted in curr 
                curr_res_color[i][j] = pj_color[i][j]
                curr_res_obj_id[i][j] = pj_obj_id[i][j]
                curr_res_global_id[i][j] = pj_global_id[i][j]


    #also save the files as needed
    #create directories (here tmp to try out & save 
    save_mask_color_dir = osp.join(data_dir, "mask", curr_scan_id, "color")
    save_mask_obj_dir = osp.join(data_dir, "mask", curr_scan_id, "obj_id")
    save_mask_global_dir = osp.join(data_dir, "mask", curr_scan_id, "global_id")

    #make sure the directories exist
    for dir_path in [save_mask_color_dir, save_mask_obj_dir, save_mask_global_dir]:
        try:
            os.makedirs(dir_path, exist_ok=True)
            
        except Exception as e:
            print(f"Failed to create directory {dir_path}: {e}")

    #save the files in the corresponding directories
        
    
    img_name = "new_val_mask_pose_in_"+ str(new_scan_id)+"_"+str(frame_number)+".jpg"
    obj_id_img_file = osp.join(save_mask_obj_dir, img_name)
    global_img_file = osp.join(save_mask_global_dir, img_name)
    color_img_file = osp.join(save_mask_color_dir, img_name)
    cv2.imwrite(obj_id_img_file, new_res_obj_id)
    cv2.imwrite(global_img_file, new_res_global_id)
    cv2.imwrite(color_img_file, new_res_color)


    img_name = "curr_val_mask_pose_in_"+ str(new_scan_id)+"_"+str(frame_number)+".jpg"
    obj_id_img_file = osp.join(save_mask_obj_dir, img_name)
    global_img_file = osp.join(save_mask_global_dir, img_name)
    color_img_file = osp.join(save_mask_color_dir, img_name)
    cv2.imwrite(obj_id_img_file, curr_res_obj_id)
    cv2.imwrite(global_img_file, curr_res_global_id)
    cv2.imwrite(color_img_file, curr_res_color)

    return 




In [223]:
#computes the mask and shows the mask
get_transformation_mask(data_dir, scenes_dir,curr_scan_id, new_scan_id, frame_number)

Since the imports are not working the next codesegment will be here so as to get the edges and nodes which are impacted by the ray intersection

In [250]:
#this method provides the list of the obj_id, global_obj ids and edges which are potentially modified in the current scene graph
#returns affected_curr_nodes, affected_curr_edges, affected_new_nodes, affected_new_edges ( not yet the global id)
def potential_nodes_and_edges(data_dir, curr_scan_id, new_scan_id, frame_number):

    #compute the different transformation files
    get_transformation_mask(data_dir, scenes_dir,curr_scan_id, new_scan_id, frame_number)

    #note to self: think about the access of the files -> is it necessary to actually store both masks ? since we have the projection already
    change_obj_new_path = os.path.join(data_dir, "mask", curr_scan_id, "obj_id","new_val_mask_pose_in_"+ str(new_scan_id)+"_"+str(frame_number)+".jpg")
    change_global_new_path = os.path.join(data_dir, "mask", curr_scan_id, "global_id","new_val_mask_pose_in_"+ str(new_scan_id)+"_"+str(frame_number)+".jpg")
    
    change_obj_curr_path = os.path.join(data_dir, "mask", curr_scan_id, "obj_id","curr_val_mask_pose_in_"+ str(new_scan_id)+"_"+str(frame_number)+".jpg")
    change_global_curr_path = os.path.join(data_dir, "mask", curr_scan_id, "global_id","curr_val_mask_pose_in_"+ str(new_scan_id)+"_"+str(frame_number)+".jpg")

    #access the current edges and object id/ current ids for the tranlation
    path = os.path.join(data_dir, "files/orig/data/" "{}.pkl".format(scan_id))
    with open(path, 'rb') as file:
        data = pickle.load(file)
        print("Keys in the data dictionary:", data.keys())
       

    obj2idx = data["object_id2idx"]
    edges = data["edges"]
    # obj_id = data["objects_id"]
    # global_id = data["global_objects_id"]
    # print("obj_id length", len(obj_id), "values", obj_id)
    # print("global_id length", len(global_id), "values", global_id)


    #load the files and convert them to numpyys
    new_obj = cv2.imread(change_obj_new_path, cv2.IMREAD_UNCHANGED)
    new_obj = np.array(new_obj)
    new_obj = new_obj.flatten()
    nonzero_new_obj = new_obj[new_obj!= 0]
    #this are the unique nodes which are affected
    affected_new_nodes = np.unique(nonzero_new_obj)
    #since edge access is in index also translate it
    affected_new_nodes_idx =  [obj2idx[obj_id] for obj_id in affected_new_nodes if obj_id in obj2idx]
    # new_global = cv2.imread(change_global_new_path, cv2.IMREAD_UNCHANGED)
    # new_global = np.array(new_global)


    curr_obj = cv2.imread(change_obj_curr_path, cv2.IMREAD_UNCHANGED)
    curr_obj = np.array(curr_obj)
    curr_obj = curr_obj.flatten()
    nonzero_curr_obj = new_obj[curr_obj!= 0]
    #this are the unique nodes which are affected
    affected_curr_nodes = np.unique(nonzero_curr_obj)
      #since edge access is in index also translateit
    affected_curr_nodes_idx =  [obj2idx[obj_id] for obj_id in affected_curr_nodes if obj_id in obj2idx]
    # curr_global = cv2.imread(change_global_curr_path, cv2.IMREAD_UNCHANGED)
    # curr_global = np.array(curr_global)



    affected_curr_edges_idx = [edge for edge in edges if edge[0] in affected_curr_nodes_idx or edge[1] in affected_curr_nodes_idx]
    affected_new_edges_idx = [edge for edge in edges if edge[0] in affected_new_nodes_idx or edge[1] in affected_new_nodes_idx]

    

    # img_height, img_width = new_obj.shape
    # for i in range(img_height):
    #     for j in range(img_width):
    #         #go only over the parts in curr which have a change
    #         if curr_obj[i][j] != 0:
    #             #get the current of the frame
    #             affected_curr_nodes.append(curr_obj[i][j])

    #         #go only over the parts in curr which have a change
    #         if new_obj[i][j] != 0:
    #             #get the current of the frame
    #             affected_new_nodes.append(new_obj[i][j])
    
               

    # #now based on affected nodes iterate once over the edges
    # for ed in edges:
    #     #visible in current frame and modified
    #     if ed[0] in affected_curr_nodes or ed[1] in affected_curr_nodes:
    #         affected_curr_edges.append(ed)
    #     #are new in the scene -> mus have come from somewhere not visible
    #     if ed[0] in affected_new_nodes or ed[1] in affected_new_nodes:
    #         affected_new_edges.append(ed)

    # return affected_curr_nodes, affected_curr_edges, affected_new_nodes, affected_new_edges



In [251]:
potential_nodes_and_edges(data_dir, curr_scan_id, new_scan_id, frame_number)

Keys in the data dictionary: dict_keys(['scan_id', 'objects_id', 'global_objects_id', 'objects_cat', 'triples', 'pairs', 'edges', 'obj_points', 'objects_count', 'edges_count', 'object_id2idx', 'object_attributes', 'edges_cat', 'rel_trans', 'root_obj_id', 'bounding_boxes', 'bow_vec_object_attr_feats', 'bow_vec_object_edge_feats'])
